# Langchain을 통해 가상 대화 상대 만들어보기

## 기본적인 AI Chat 만들기

In [1]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [2]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [3]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_60801/3353824168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [4]:
conv_chain = (  # {"input": "hello"}
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
    ) # {"input": "hello", "chat_history": [~~~]}
    | prompt
    | model
)

### Runnable 이해하기

#### LCEL(LangChain Expression Language)?
- LangChain Expression Language, 또는 LCEL,은 체인을 쉽게 결합할 수 있는 선언적 방법입니다. LCEL은 가장 간단한 "프롬프트 + LLM" 체인부터 수만은 체인을 쉽게 연결 할 수 있도록 도와줍니다.
- https://python.langchain.com/docs/expression_language/why
- LCEL object 는 Runnable Interface를 구현함.

#### Runnable Interface
- Runnable Interface는 invoke, batch, stream, ainvoke 등을 구현해야함
- Runnable은 '|'를 이용해 Chain으로 연결 할 수 있음

#### RunnablePassthrough ?
- RunnablePassthrough는 입력을 그대로 전달하거나 추가 키를 추가하여 전달할 수 있습니다. 이는 일반적으로 RunnableParallel과 함께 사용되어 맵에 새 키에 데이터를 할당합니다.
- RunnablePassthrough()는 단독으로 호출될 경우, 입력을 그대로 받아서 전달합니다.
- RunnablePassthrough가 assign 함수와 함께 호출되면 (RunnablePassthrough.assign(...)), 입력을 받아서 assign 함수에 전달된 추가 인자를 추가합니다.

In [5]:
# RunnablePassthrough Example

from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

### 다시 예제를 보면

```python
chain = (  # {"input": "hello"}
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
    ) # {"input": "hello", "chat_history": [~~~]}
    | prompt
    | model
)
```

In [6]:
memory.load_memory_variables({})  # Runnable이 아님

{'chat_history': []}

In [7]:
# memory.load_memory_variables를 Runnable로 만들고 
get_memory_chain = RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")

In [8]:
get_memory_chain.invoke({})

[]

In [9]:
inputs = {"input": "안녕"}
response = conv_chain.invoke(inputs)
response

AIMessage(content='안녕하세요! 도움이 필요하신 것이 있나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 21, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMBSMqUSOyli0dx3C1Rfvkwz830FF', 'finish_reason': 'stop', 'logprobs': None}, id='run-6bb7b1ad-302a-438a-82b7-b349f7ec32fb-0', usage_metadata={'input_tokens': 21, 'output_tokens': 22, 'total_tokens': 43, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
memory.load_memory_variables({})

{'chat_history': []}

In [11]:
memory.save_context(inputs, {"output": response.content})

In [12]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='안녕', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요! 도움이 필요하신 것이 있나요?', additional_kwargs={}, response_metadata={})]}

In [13]:
quit_msg = {'q', ''}
while True:
    user_input = input("user: ")
    if user_input in quit_msg:
        break
        
    ai_output = conv_chain.invoke({"input": user_input})
    print(f"AI: {ai_output.content}")
    memory.save_context({"input": user_input}, {"output": ai_output.content})

AI: Hello! How can I help you today?


## 인격 부여하기

In [14]:
system_prompt = """\
- 너는 20대 여성 AI 개발자이다.
- 처음 만나는 1:1 소개팅 상황이다. 커피집에서 만났다.
- 소개팅이기에 너무 도움을 주려고 대화하지 않는다. 자연스러운 대화를한다.
- 너무 적극적으로 이야기하지 않는다.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [15]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [16]:
conv_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
    )
    | prompt
    | model
)

In [17]:
quit_msg = {'q', ''}
while True:
    user_input = input("user: ")
    if user_input in quit_msg:
        break
    ai_output = conv_chain.invoke({"input": user_input})
    print(f"AI: {ai_output.content}")
    memory.save_context({"input": user_input}, {"output": ai_output.content})

AI: 안녕하세요. 만나서 반가워요. 혹시 커피는 드셔보셨나요?


## Streaming

- streaming은 AI의 응답이 다 완성되기 이전에도 응답을 들을 수 있는 방법
- 사용자가 기다리는 시간을 줄여줌
- LCEL object들은 stream이 구현되어 있음

### ChatModel의 Stream

In [18]:
import time

In [19]:
for chunk in model.stream("안녕"):
    time.sleep(0.5)
    print(chunk.content, end="", flush=True)

안녕하세요! 도와드릴 수 있는 게 있나요? :)

### Chain의 Stream

In [20]:
inputs = {'input': '안녕'}

In [21]:
# conv_chain도 LCEL object이기 때문에 stream이 구현되어있음
for chunk in conv_chain.stream(inputs):
    print(chunk.content, end="", flush=True)

안녕하세요. 오늘은 날씨가 좋네요. 요즘 할 일이 있나요?

### Stream을 활용한 대화

In [22]:
quit_msg = {'q', ''}
while True:
    user_input = input("user: ")
    if user_input in quit_msg:
        break
    print(f"AI: ", end='')
    response = ""
    for chunk in conv_chain.stream({"input": user_input}):
        print(chunk.content, end="", flush=True)
        response += chunk.content
    memory.save_context({"input": user_input}, {"output": response})